# Day 4: Kalman Filter + Dual-Axle PID with OpenCV HUD


## 🎯 Learning Goals
1. Integrate Kalman Filter for sideslip estimation with Dual-Axle PID control.  
2. Display real-time telemetry (HUD) using OpenCV.  
3. Compare KF-enhanced vs raw PID lane keeping in a simulated loop.



## 1. Why HUD?
While Matplotlib plots help analyze results **after simulation**,  
a HUD overlay shows **real-time feedback** like in ADAS dashboards.

### HUD Will Display:
- Vehicle speed (constant in this sim)  
- Estimated vs true sideslip angle ($\beta$)  
- Lane offset (measured & corrected)  
- Steering command  


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import cv2

# Parameters
dt = 0.1
T = 20
time = np.arange(0, T, dt)
vx = 15.0  # forward speed [m/s]

# Simulated true dynamics
true_offset = np.sin(0.2*time)
true_vy = np.gradient(true_offset, dt)
true_r = 0.05*np.cos(0.2*time)
true_beta = np.arctan2(true_vy, vx)

# Noisy measurements
meas_offset = true_offset + np.random.normal(0, 0.05, len(time))
meas_r = true_r + np.random.normal(0, 0.01, len(time))

# Kalman Filter init
A = np.array([[1, dt],
              [0, 1]])
H = np.eye(2)
Q = np.array([[0.001, 0],
              [0, 0.001]])
R = np.array([[0.05, 0],
              [0, 0.01]])
x_est = np.array([0, 0])
P = np.eye(2)

# PID params
Kp, Ki, Kd = 0.5, 0.02, 0.1
integral, prev_error = 0, 0

# Logs
offsets_pid, offsets_kf, betas_est, betas_true, steering_cmds = [], [], [], [], []

# OpenCV HUD window
for k in range(len(time)):
    # KF Prediction
    x_pred = A @ x_est
    P_pred = A @ P @ A.T + Q
    z = np.array([np.gradient(meas_offset, dt)[k], meas_r[k]])
    K = P_pred @ H.T @ np.linalg.inv(H @ P_pred @ H.T + R)
    x_est = x_pred + K @ (z - H @ x_pred)
    P = (np.eye(2) - K @ H) @ P_pred

    vy_est, r_est = x_est
    beta_est = np.arctan2(vy_est, vx)

    # PID control
    error = meas_offset[k]
    integral += error * dt
    derivative = (error - prev_error) / dt
    steer_cmd_f = Kp*error + Ki*integral + Kd*derivative

    if vx < 8:
        k_rear = -0.5
    else:
        k_rear = 0.3 - 0.1*abs(beta_est)
    steer_cmd_r = k_rear * steer_cmd_f
    combined_steer = (steer_cmd_f + steer_cmd_r) / 2

    # Store logs
    offsets_pid.append(error)
    offsets_kf.append(error - beta_est)
    betas_est.append(beta_est)
    betas_true.append(true_beta[k])
    steering_cmds.append(combined_steer)
    prev_error = error

    # HUD frame
    hud = np.zeros((300, 600, 3), dtype=np.uint8)
    cv2.putText(hud, f"Time: {time[k]:.1f}s", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"Speed: {vx*3.6:.1f} km/h", (20,80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"Lane Offset: {error:.2f} m", (20,120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"Corrected Offset: {offsets_kf[-1]:.2f} m", (20,160), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"True β: {true_beta[k]:.3f} rad", (20,200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"Estimated β: {beta_est:.3f} rad", (20,240), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    cv2.putText(hud, f"Steering Cmd: {combined_steer:.3f}", (20,280), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

    cv2.imshow("KF + Dual-Axle PID HUD", hud)
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# Final plots
plt.figure(figsize=(14,8))
plt.subplot(2,2,1)
plt.plot(time, true_offset, label="True Lane Offset")
plt.plot(time, meas_offset, label="Measured Offset (noisy)")
plt.xlabel("Time (s)"); plt.ylabel("Offset (m)"); plt.title("Lane Offset Tracking")
plt.legend(); plt.grid(True)

plt.subplot(2,2,2)
plt.plot(time, betas_true, label="True β")
plt.plot(time, betas_est, label="Estimated β (KF)")
plt.xlabel("Time (s)"); plt.ylabel("Sideslip Angle (rad)"); plt.title("Sideslip Estimation")
plt.legend(); plt.grid(True)

plt.subplot(2,2,3)
plt.plot(time, offsets_pid, label="PID Offset (no KF)")
plt.plot(time, offsets_kf, label="Corrected Offset (KF)")
plt.xlabel("Time (s)"); plt.ylabel("Offset (m)"); plt.title("PID vs KF Offset")
plt.legend(); plt.grid(True)

plt.subplot(2,2,4)
plt.plot(time, steering_cmds, label="Steering Command")
plt.xlabel("Time (s)"); plt.ylabel("Steer (arb units)"); plt.title("Dual-Axle Steering Command")
plt.legend(); plt.grid(True)

plt.tight_layout()
plt.show()



## 4. Observations

- The HUD showed **real-time telemetry**: speed, lane offset, sideslip, and steering.  
- KF improved stability by reducing corrected lane offset.  
- Plots confirm smoother control with KF integration.

### ✅ Conclusion
OpenCV HUD brings real-time ADAS feedback, complementing post-analysis plots.  
This mirrors actual in-vehicle systems, bridging simulation with ADAS design.
